In [1]:
import pandas as pd
import pymongo
import requests
import datetime
import jieba
import jieba.analyse
import base64
import random
import time
import zlib
import json

In [2]:
URL = 'https://wh.meituan.com/meishi/'
_cookies = ['_lxsdk_s=b458c64cc9ef92adce92c199a6b7%7C%7C4; __mta=214784936.1512623224439.1512623224439.1512623226558.2; client-id=8d050bd8-51b6-46fc-be96-cb6871d91fe7; ci=89; webloc_geo=31.778615%2C119.958931%2Cwgs84; _lxsdk=1602f5eb154c8-05c63a41825c168-1b451e24-13c680-1602f5eb154c8; uuid=73a65301-f351-4cdd-bfcd-35ae40aea13c',
           '_lxsdk_s=55618265813fc43043ce562acf29%7C%7C2; ci=89; __mta=252597174.1512623399814.1512623399814.1512623399814.1; _lxsdk=1602f6161c1c8-034ae66c3842d88-1b451e24-13c680-1602f6161c170; client-id=8ca2fe5f-ba96-41ae-b919-f8d3665e2f10; webloc_geo=31.778620%2C119.958935%2Cwgs84; uuid=ae73c608-8db3-4724-b4bf-91f4a8c701c5',
           '_lxsdk_s=4ffb12d5a90c2361745e6d0993a2%7C%7C2; ci=89; __mta=40613533.1512623465821.1512623465821.1512623465821.1; client-id=a31f1c79-3287-4ef1-8bd3-56bfb431a7da; webloc_geo=31.778616%2C119.958936%2Cwgs84; _lxsdk=1602f6262117d-0f173d7fde38d18-1b451e24-13c680-1602f626212c8; uuid=ba1b457e-ae34-4b56-bf7b-9bf3579ac0ae',
           '_lxsdk_s=4ba654e682eaf4d69efb90ae9cf1%7C%7C2; __mta=152351622.1512623525333.1512623525333.1512623525333.1; ci=89; client-id=e8101f58-a778-4332-81c2-75ba410b2bb3; webloc_geo=31.778611%2C119.958934%2Cwgs84; _lxsdk=1602f634c448d-0c74adf6d00e2e8-1b451e24-13c680-1602f634c45c8; uuid=e98a7e32-7511-4b40-ab3d-d88011a06471',
           '_lxsdk_s=f6511ca715103c33b7bb4aea1e9f%7C%7C2; __mta=40629917.1512623573270.1512623573270.1512623573270.1; ci=89; client-id=c69db65e-b5d1-4fe6-aa7a-340e3608aeb6; webloc_geo=31.778617%2C119.958930%2Cwgs84; _lxsdk=1602f64072fc8-04cf33034885b08-1b451e24-13c680-1602f640730c8; uuid=33b52a9f-14ea-4f80-9a97-8386fc9fb0a2',
           '_lxsdk_s=38327d7cfda78d0cfc4810838c32%7C%7C2; __mta=150795529.1512623625315.1512623625315.1512623625315.1; ci=89; client-id=221306b1-bfb7-4cfa-ab3f-2e4a85089864; webloc_geo=31.778610%2C119.958933%2Cwgs84; _lxsdk=1602f64d24531-0623a27320ba75-1b451e24-13c680-1602f64d246c8; uuid=85f7949c-284c-4ae6-b5f8-18aa1c6dc016',
           '_lxsdk_s=c2f111bde916cad4d653e07a35c9%7C%7C2; __mta=42950426.1512623702441.1512623702441.1512623702441.1; _lxsdk=1602f66000dc8-0bb60c750cf17b8-1b451e24-13c680-1602f66000d7f; ci=89; client-id=38cc3a86-fe57-491b-8564-c9b1a251cd4a; webloc_geo=31.778617%2C119.958931%2Cwgs84; uuid=095c6c87-a397-4cc4-a3fc-1c3e35eca3be',
           '_lxsdk_s=fd711080efee1adf6de57c1d4c88%7C%7C2; ci=89; webloc_geo=31.778612%2C119.958939%2Cwgs84; __mta=218070313.1512623752950.1512623752950.1512623752950.1; _lxsdk=1602f66c53e35-0b363d23b0e435-1b451e24-13c680-1602f66c53fc8; client-id=1a4a2af3-4201-4e61-bf26-92016709444b; uuid=9361e591-ada9-43c0-a762-b7b5559c649a',
           '_lxsdk_s=810677247f21b53cd75826e60cb8%7C%7C2; __mta=146594315.1512623869611.1512623869611.1512623869611.1; _lxsdk=1602f688ce80-074a1ec97ff043-1b451e24-13c680-1602f688ce9c8; ci=89; client-id=e60488fa-51fe-4d12-99a3-257a5a0853cc; webloc_geo=31.778609%2C119.958928%2Cwgs84; uuid=948eec52-7d10-46ce-b689-92140842f3f0',
           '_lxsdk_s=c44c26606c13c651ab5a20447428%7C%7C2; __mta=89516452.1512623906613.1512623906613.1512623906613.1; _lxsdk=1602f691da0c8-024ebcb15833bc-1b451e24-13c680-1602f691da04b; ci=89; client-id=a6846506-a50a-4612-8106-ff31978802a8; webloc_geo=31.778607%2C119.958929%2Cwgs84; uuid=6ee14a07-2d20-460e-87c2-001806dfdf1b',
           '_lxsdk_s=071dfe26ce64d6e46d03323d2f71%7C%7C2; __mta=210099498.1512623985254.1512623985254.1512623985254.1; ci=89; client-id=f2cea0fa-0d83-479a-a863-f2b7156fb392; webloc_geo=31.778621%2C119.958925%2Cwgs84; _lxsdk=1602f6a4f88c8-0ce1602d9c121a-1b451e24-13c680-1602f6a4f88c8; uuid=122a1bf8-f588-4b2d-a509-2bb45aaf1be9',
           '_lxsdk_s=833017bd50026c6c57b327f1f51a%7C%7C2; __mta=244702778.1512624031536.1512624031536.1512624031536.1; ci=89; client-id=ec7106e4-9dda-4ce0-aa8d-c12288812644; webloc_geo=31.778622%2C119.958921%2Cwgs84; _lxsdk=1602f6b0459c8-011b3f8b51c9328-1b451e24-13c680-1602f6b045ac8; uuid=04d6dbed-d679-4e9d-a0c7-4ea1b2343513',
           '_lxsdk_s=9a68aaba42beb1ebdfc3e726d94e%7C%7C2; ci=89; webloc_geo=31.778629%2C119.958912%2Cwgs84; __mta=45406747.1512624277952.1512624277952.1512624277952.1; _lxsdk=1602f6ec7b84-095c2ecf9438458-1b451e24-13c680-1602f6ec7b9c8; uuid=33acafd1-4a03-4f0f-9572-1ccbfd570efc; client-id=77353c1f-ba56-4ac9-bbb3-c7a689858182',
           '_lxsdk_s=c1508df449b0616ba37c5d9166f2%7C%7C2; __mta=216814121.1512624317347.1512624317347.1512624317347.1; ci=89; client-id=5d8e3ff2-5370-4c08-855e-9d26e5187040; webloc_geo=31.778633%2C119.958912%2Cwgs84; _lxsdk=1602f6f6187a2-029abec138ea508-1b451e24-13c680-1602f6f6188c8; uuid=1a808ef4-511d-44ba-acfd-d6339ddb92f8',
           '_lxsdk_s=7a249ecc9a048cbb826ea3b1f4e7%7C%7C2; __mta=256753588.1512624355644.1512624355644.1512624355644.1; _lxsdk=1602f6ff822c8-01f025c7119f968-1b451e24-13c680-1602f6ff823c8; ci=89; client-id=33339c7c-4dd0-4da8-8617-add389e72438; webloc_geo=31.778634%2C119.958912%2Cwgs84; uuid=92e531c6-4620-453e-ad8d-0e67644404ad',
           '_lxsdk_s=93f6847b5f37e552ae083beb48c2%7C%7C2; __mta=218397606.1512624389136.1512624389136.1512624389136.1; ci=89; client-id=21540b20-0240-48fa-ae1e-c1ce4dba892f; webloc_geo=31.778631%2C119.958925%2Cwgs84; _lxsdk=1602f707a3d3-0d7db668e6d9498-1b451e24-13c680-1602f707a3ec8; uuid=507d006a-6b37-4ff9-9299-88eae1bc0982',
           '_lxsdk_s=0ffdd4e2dddd57d3c05606eb82f3%7C%7C2; __mta=141367821.1512624428164.1512624428164.1512624428164.1; ci=89; client-id=d6e8de67-48b8-4262-b3b3-47851c46db5c; webloc_geo=31.778620%2C119.958977%2Cwgs84; _lxsdk=1602f7112b9c8-09d6e858772c1d8-1b451e24-13c680-1602f7112bac8; uuid=2b6ac2bf-62cf-4ebd-bb68-bd0c258c6b29',
           ]
cookie = '_lxsdk_cuid=16a2f66fef2c8-04adb68a07ed52-e323069-130980-16a2f66fef2c8; ci=57; rvct=57; _hc.v=1197cd1c-6791-15f8-9279-cd9a57fde16c.1555573509; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; _lxsdk=16a2f66fef2c8-04adb68a07ed52-e323069-130980-16a2f66fef2c8; __mta=251043129.1555573459093.1555902899880.1555913774718.3; client-id=dc69c296-2418-478e-bb1c-b5dc4676a0b5; uuid=388dc241-38f9-4e12-bc70-c9898b7c1c1c; _lxsdk_s=16a48f9f2c4-2f-01c-78c%7C%7C38'
headers = {
  'Accept':'application/json',
  'Accept-Encoding': 'gzip, deflate',
  'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
  'Connection': 'keep-alive',
  'Cookie': cookie,
  'Host': 'wh.meituan.com',
  'Referer': 'http://wh.meituan.com/meishi/',
  'User-Agent':' Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}
URL = 'https://wh.meituan.com/meishi/api/poi/'

In [3]:
# Constant
client = pymongo.MongoClient("mongodb://101.132.171.223:27017/")
db = client['shoprec']
totalCount = 1000
pageNum = 67

In [4]:
class Token():
    def __init__(self, cateId, areaId, originUrl, page):
        self.areaId = areaId
        self.cityName = '武汉'
        self.originUrl = originUrl
        self.page = page
        self.uuid = '765ddffde8444259bfe4.1556036967.1.0.0'
        self.cateId = cateId

    def join_sign(self):
        # 参数
        sign = 'cityName={cityName}&areaId={areaId}&cateId={cateId}&dinnerCountAttrId=&optimusCode=1&originUrl={originUrl}&page={page}&partner=126&platform=1&riskLevel=1&sort=&userId=&uuid={uuid}'
        _str = sign.format(areaId=self.areaId, cateId=self.cateId, cityName=self.cityName, originUrl=self.originUrl, page=self.page,
                           uuid=self.uuid)
        sign = base64.b64encode(zlib.compress(bytes(json.dumps(_str, ensure_ascii=False), encoding="utf8")))
        sign = str(sign, encoding="utf8")
        return sign

    @property
    def join_token(self):
        str_json = {}
        str_json['rId'] = 100900
        str_json['ver'] = '1.0.6'
        str_json['ts'] = time.time()
        str_json['cts'] = time.time() + 110
        str_json['brVD'] = [1920, 315]
        str_json['brR'] = [[1920, 1080], [1920, 1057], 24, 24]
        str_json['bI'] = [self.originUrl, ""]
        str_json['mT'] = []
        str_json['kT'] = []
        str_json['aT'] = []
        str_json['tT'] = []
        str_json['aM'] = ''
        str_json['sign'] = self.join_sign()
        token_decode = zlib.compress(
            bytes(json.dumps(str_json, separators=(',', ':'), ensure_ascii=False), encoding="utf8"))
        token = str(base64.b64encode(token_decode), encoding="utf8")
        return token

In [5]:
def getPoiList(cateId, areaId, page, originUrl):
    url = URL + 'getPoiList'
    _token = Token(cateId, areaId, page, originUrl)
    _params = {
        'cityName': '武汉',
        'cateId': cateId,
        'areaId': areaId,
        'page': page,
        'uuid': '765ddffde8444259bfe4.1556036967.1.0.0',
        'originUrl': originUrl,
        '_token': _token.join_token,
        'platform': 1,
        'partner': 126,
        'riskLevel': 1,
        'optimusCode': 1,
        'sort':'',
        'dinnerCountAttrId':'',
        'userId':''
    }
    content = requests.get(url, headers=headers, params=_params)
    return json.loads(content.text, encoding='utf8')

def getType():
    col = db['cate']
    types = col.find()
    return list(types)

def getPoiByCate(cate):
    name = cate['name']
    cateId = cate['cateId']
    url = cate['url']
    
    shops = []
    areaId = 0
    for page in range(1, pageNum + 1):
        originUrl = URL + url + '/pn' + str(page)
        res = getPoiList(cateId, areaId, page, originUrl)
        data = res['data']['poiInfos']
        data = list(map(lambda x: {
            'id': x['poiId'],
            'title': x['title'],
            'address': x['address'],
            'avgPrice': x['avgPrice'],
            'avgScore': x['avgScore'],
            'allCommentNum': x['allCommentNum'],
            'cate': name,
            'cateId': cateId
        }, data))
        shops += data
    return shops

In [113]:
_types = list(map(lambda x: {'name': x['name'], 'cateId': x['cateId'][1:], 'url': x['cateId']}, getType()))

In [125]:
for cate in _types:
    data = getPoiByCate(cate)
    print("Getting...", cate['name'], ", Total count = ", len(data))
    col.insert_many(data)
    print("Inserting data...")

Getting... 蛋糕甜点 , Total count =  1005
Inserting data...
Getting... 火锅 , Total count =  1005
Inserting data...
Getting... 自助餐 , Total count =  189
Inserting data...
Getting... 小吃快餐 , Total count =  1005
Inserting data...
Getting... 日韩料理 , Total count =  391
Inserting data...
Getting... 聚餐宴请 , Total count =  656
Inserting data...
Getting... 西餐 , Total count =  562
Inserting data...
Getting... 东北菜 , Total count =  26
Inserting data...
Getting... 烧烤烤肉 , Total count =  885
Inserting data...
Getting... 川湘菜 , Total count =  1005
Inserting data...
Getting... 香锅烤鱼 , Total count =  279
Inserting data...
Getting... 江浙菜 , Total count =  28
Inserting data...
Getting... 中式烧烤/烤串 , Total count =  659
Inserting data...
Getting... 粤菜 , Total count =  119
Inserting data...
Getting... 咖啡酒吧 , Total count =  424
Inserting data...
Getting... 西北菜 , Total count =  20
Inserting data...
Getting... 京菜鲁菜 , Total count =  41
Inserting data...
Getting... 云贵菜 , Total count =  20
Inserting data...
Getting... 东南亚菜 , To

In [6]:
commentUrl = 'http://www.meituan.com/meishi/api/poi/getMerchantComment'
url = 'http://www.meituan.com/meishi/'
uuid = 'c463a63811ee45109b36.1556376775.1.0.0'
proxies = {
    "https": "http://111.177.178.164:9999", 
    "http": "http://112.85.131.3:9999"
}
def getLastComment(x):
    _id = x['id']
    num = x['allCommentNum']
    referer = url + str(_id)
    _headers = {
    'Accept':'application/json',
    'Referer': referer,
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
    }
    r = requests.get(commentUrl, params={
        'uuid': uuid,
        'platform': 1,
        'partner': 126,
        'originUrl': referer,
        'riskLevel': 1,
        'optimusCode': 1,
        'id': _id,
        'userId':'',
        'offset': num - 1,
        'pageSize': 1,
        'sortType': 1
    }, headers=_headers)
    j = json.loads(r.text, encoding='utf8')
    j = j['data']['comments']
    if j == None:
        x['year'] = 2019
        x['month'] = 5
        x['day'] = 1
        x['duration'] = 0
        return x;
    j = j[-1]
    _time = time.localtime(int(j['commentTime']) / 1000)
    x['year'] = _time.tm_year
    x['month'] = _time.tm_mon
    x['day'] = _time.tm_mday
    now = datetime.datetime(2019, 5, 1)
    x['duration'] = (now - datetime.datetime(_time.tm_year, _time.tm_mon, _time.tm_mday)).days
    return x

In [7]:
_col = db['type']
data = list(_col.find())

In [9]:
# res = []

In [15]:
for i in range(8198, len(data)):
    _res = getLastComment(data[i])
    res.append(_res)
    if i % 50 == 0:
        time.sleep(random.randint(3, 5))
    print("Getting...", i, len(data))

Getting... 8198 9022
Getting... 8199 9022
Getting... 8200 9022
Getting... 8201 9022
Getting... 8202 9022
Getting... 8203 9022
Getting... 8204 9022
Getting... 8205 9022
Getting... 8206 9022
Getting... 8207 9022
Getting... 8208 9022
Getting... 8209 9022
Getting... 8210 9022
Getting... 8211 9022
Getting... 8212 9022
Getting... 8213 9022
Getting... 8214 9022
Getting... 8215 9022
Getting... 8216 9022
Getting... 8217 9022
Getting... 8218 9022
Getting... 8219 9022
Getting... 8220 9022
Getting... 8221 9022
Getting... 8222 9022
Getting... 8223 9022
Getting... 8224 9022
Getting... 8225 9022
Getting... 8226 9022
Getting... 8227 9022
Getting... 8228 9022
Getting... 8229 9022
Getting... 8230 9022
Getting... 8231 9022
Getting... 8232 9022
Getting... 8233 9022
Getting... 8234 9022
Getting... 8235 9022
Getting... 8236 9022
Getting... 8237 9022
Getting... 8238 9022
Getting... 8239 9022
Getting... 8240 9022
Getting... 8241 9022
Getting... 8242 9022
Getting... 8243 9022
Getting... 8244 9022
Getting... 82

Getting... 8589 9022
Getting... 8590 9022
Getting... 8591 9022
Getting... 8592 9022
Getting... 8593 9022
Getting... 8594 9022
Getting... 8595 9022
Getting... 8596 9022
Getting... 8597 9022
Getting... 8598 9022
Getting... 8599 9022
Getting... 8600 9022
Getting... 8601 9022
Getting... 8602 9022
Getting... 8603 9022
Getting... 8604 9022
Getting... 8605 9022
Getting... 8606 9022
Getting... 8607 9022
Getting... 8608 9022
Getting... 8609 9022
Getting... 8610 9022
Getting... 8611 9022
Getting... 8612 9022
Getting... 8613 9022
Getting... 8614 9022
Getting... 8615 9022
Getting... 8616 9022
Getting... 8617 9022
Getting... 8618 9022
Getting... 8619 9022
Getting... 8620 9022
Getting... 8621 9022
Getting... 8622 9022
Getting... 8623 9022
Getting... 8624 9022
Getting... 8625 9022
Getting... 8626 9022
Getting... 8627 9022
Getting... 8628 9022
Getting... 8629 9022
Getting... 8630 9022
Getting... 8631 9022
Getting... 8632 9022
Getting... 8633 9022
Getting... 8634 9022
Getting... 8635 9022
Getting... 86

Getting... 8980 9022
Getting... 8981 9022
Getting... 8982 9022
Getting... 8983 9022
Getting... 8984 9022
Getting... 8985 9022
Getting... 8986 9022
Getting... 8987 9022
Getting... 8988 9022
Getting... 8989 9022
Getting... 8990 9022
Getting... 8991 9022
Getting... 8992 9022
Getting... 8993 9022
Getting... 8994 9022
Getting... 8995 9022
Getting... 8996 9022
Getting... 8997 9022
Getting... 8998 9022
Getting... 8999 9022
Getting... 9000 9022
Getting... 9001 9022
Getting... 9002 9022
Getting... 9003 9022
Getting... 9004 9022
Getting... 9005 9022
Getting... 9006 9022
Getting... 9007 9022
Getting... 9008 9022
Getting... 9009 9022
Getting... 9010 9022
Getting... 9011 9022
Getting... 9012 9022
Getting... 9013 9022
Getting... 9014 9022
Getting... 9015 9022
Getting... 9016 9022
Getting... 9017 9022
Getting... 9018 9022
Getting... 9019 9022
Getting... 9020 9022
Getting... 9021 9022


In [16]:
len(res)

4200

In [17]:
_res = [dict(t) for t in set([tuple(d.items()) for d in res])]

In [18]:
df = pd.io.json.json_normalize(_res)

In [19]:
df.to_excel('restautant1.xlsx')

In [20]:
_col = db['restaurant']
_col.insert_many(_res)